In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime
import time
from tqdm import tqdm
import sys
import glob

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [52]:
submit_path = '0215_083_submit_outlier_processing_blend_base1_clf_out1_rm_out1_no_out_blend0_minthres0.01_CV3.6105891261264405_LB3.664.csv'
pred = 'prediction'
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz').set_index(key)[['clf_pred', target, 'no_out_flg', 'first_active_month']]
submit = pd.read_csv('../manual/' + submit_path).set_index(key)
# submit = pd.read_csv('../log_submit/0218_005_submit_lgb_lr0.01_279feats_10seed_70leaves_iter1177_OUT29.8158_CV3.6180025980533452_LB.csv').set_index(key)
base[pred] = submit[target]

In [43]:
clf = utils.read_pkl_gzip('../stack/0218_075_stack_lgb_binary_valid-ods_seed3_CV0-9098440141342908_235features.gz').set_index(key)['pred_mean']
base['clf_pred_2'] = clf

In [10]:
# hist load
df_hist = pd.read_csv('../input/historical_transactions.csv')
df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)
use_cols = [key, 'merchant_id', 'month_lag', 'purchase_date', 'purchase_amount_new']
df_hist = df_hist[use_cols]
df_hist.sort_values(by=[key, 'purchase_date'], inplace=True)

In [48]:
# base.head(100)
base = base[base[target].isnull()]
base.sort_values(by='clf_pred', ascending=False).head(30)

,clf_pred,target,no_out_flg,first_active_month,prediction
card_id,,,,,
C_ID_ac114ef831,0.664184,NaN,0.0,2016-12-01,-24.253229
C_ID_922f9c5ea6,0.596950,NaN,0.0,2014-10-01,-20.353062
C_ID_767923bdb9,0.595777,NaN,0.0,2016-09-01,-22.081786
C_ID_63d38cafcb,0.569745,NaN,0.0,2016-12-01,-16.377531
C_ID_74dd007aec,0.561677,NaN,0.0,2017-02-01,-16.610205
C_ID_7598b04bdf,0.557790,NaN,0.0,2016-12-01,-20.538855
C_ID_938dec7a1a,0.541546,NaN,0.0,2017-05-01,-19.719827
C_ID_70c457436a,0.522289,NaN,0.0,2016-03-01,-15.588247
C_ID_9241ae9fc5,0.518362,NaN,0.0,2016-11-01,-17.445786


In [51]:
# base['clf_pred_bin'] = base['clf_pred'].map(lambda x: np.round(x, 2))
# base.groupby(['clf_pred_bin'])[pred, target].mean()

In [65]:
#========================================================================
# Deep Dive
"""
C_ID_1afd4a6a1f: ---> Public -33 is True
C_ID_32734f4f7c: ---> Private
C_ID_b311c79013: ---> Private
"""
#========================================================================

uid = 'C_ID_b311c79013'
# df_hist[df_hist[key]==uid]
submit.loc[submit.index==uid]

,target
card_id,
C_ID_b311c79013,-18.418188


In [68]:
#========================================================================
# Manual Setting 
# Be Carefull!!
# 毎回ロードしないと、マニュアルのままになる
submit_path = '0215_083_submit_outlier_processing_blend_base1_clf_out1_rm_out1_no_out_blend0_minthres0.01_CV3.6105891261264405_LB3.664.csv'
submit = pd.read_csv('../manual/' + submit_path).set_index(key)
#========================================================================

#========================================================================
# -33確定のcard_id
submit.loc[submit.index=='C_ID_1afd4a6a1f'] = -33.219281
#========================================================================

# Setting
submit.loc[submit.index==uid] = -33.219281
submit.loc[submit.index==uid]

,target
card_id,
C_ID_b311c79013,-33.219281


In [67]:
submit.to_csv(f'../submit/{submit_path}_MANUAL_{uid}=-33')